In [1]:
%%writefile mapper_tf_idf.py
#!/usr/bin/env python2
import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

with open('stop_words_en.txt', 'r') as stop_file:
    stop_words_str = stop_file.read()
stop_words_list = re.split('\n', stop_words_str, flags=re.UNICODE)

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue

    text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
    words = map(lambda x: x.lower(), re.split("\W*\s+\W*", text, flags=re.UNICODE))
        
    if article_id == '12':
        laborNumb = 0
        otherNumb = 0
        for word in words:
            if word not in stop_words_list:
                if word != 'labor':
                    otherNumb += 1
                else:
                    laborNumb += 1
        print "%s\t%d\t%f" % (article_id, 1, (float(laborNumb)/otherNumb))
    else:
        if 'labor' in words:
            print "%s\t%d" % (article_id, 1)

Writing mapper_tf_idf.py


In [2]:
%%writefile reducer_tf_idf.py
#!/usr/bin/env python2
import sys
import math

docNumber = 0
tfVal = 0

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        if key == '12':
            key, count, tfVal = line.strip().split('\t', 2)
            print 'tf\t%s' % tfVal
            
        count = int(count)
    except ValueError as e:
        continue
    docNumber += count
    
print "doc\t%d" % docNumber


Writing reducer_tf_idf.py


In [7]:
%%writefile reducer_tf_idf_final.py
#!/usr/bin/env python2
import sys
import math

docNumber = 0
tfVal = 0

for line in sys.stdin:
    try:
        key, val = line.strip().split('\t', 1)
        
        if key == 'tf':
            val = float(val)
            tfVal = val
        elif key == 'doc':
            val = int(val)
            docNumber += val
    except ValueError as e:
        continue
    
print "%f" % (tfVal * (float(1)/math.log(1+docNumber)))


Overwriting reducer_tf_idf_final.py


# App launcher

In [8]:
%%bash
TF_IDF="tf"
FINAL="final"

hdfs dfs -rm -r -skipTrash ${TF_IDF} > /dev/null
hdfs dfs -rm -r -skipTrash ${FINAL} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.job.name="Streaming_calc_TF_IDF" \
    -D mapreduce.job.reduces=2 \
    -files mapper_tf_idf.py,/datasets/stop_words_en.txt,reducer_tf_idf.py \
    -mapper "python mapper_tf_idf.py" \
    -reducer "python reducer_tf_idf.py" \
    -input /data/wiki/en_articles_part \
    -output ${TF_IDF} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.job.name="Streaming_calc_TF_IDF_final" \
    -D mapreduce.job.reduces=1 \
    -files reducer_tf_idf_final.py \
    -mapper "cat" \
    -reducer "python reducer_tf_idf_final.py" \
    -input ${TF_IDF}/part-0000* \
    -output ${FINAL} > /dev/null

hdfs dfs -cat ${FINAL}/part-0000* | head  

0.000351	


17/12/06 21:18:15 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/12/06 21:18:15 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
17/12/06 21:18:16 INFO mapred.FileInputFormat: Total input files to process : 1
17/12/06 21:18:16 INFO mapreduce.JobSubmitter: number of splits:2
17/12/06 21:18:16 INFO Configuration.deprecation: mapred.job.name is deprecated. Instead, use mapreduce.job.name
17/12/06 21:18:16 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1512506126399_0004
17/12/06 21:18:16 INFO impl.YarnClientImpl: Submitted application application_1512506126399_0004
17/12/06 21:18:16 INFO mapreduce.Job: The url to track the job: http://00a42943adf0:8088/proxy/application_1512506126399_0004/
17/12/06 21:18:16 INFO mapreduce.Job: Running job: job_1512506126399_0004
17/12/06 21:18:22 INFO mapreduce.Job: Job job_1512506126399_0004 running in uber mode : false
17/12/06 21:18:22 INFO mapreduce.Job:  map 0% reduce 0%
17/12/06 21:18:35 INFO m